In [14]:
from feature_extractor import *
extractor = SIFT()
image = cv2.imread('Untitled.png', cv2.IMREAD_COLOR)
kp, des = extractor.extract(image)
print(len(kp))
print(des.shape)

389
(389, 128)


In [2]:
from feature_extractor import HuMoments
import cv2

extractor = HuMoments()
image = cv2.imread('Untitled.png', cv2.IMREAD_COLOR)
features = extractor.extract(image)
print(features)
image = cv2.imread('Untitled.png', cv2.IMREAD_GRAYSCALE)
features = extractor.extract(image)
print(features)

[  2.57727183   5.74679831  10.58957531  10.05163076  20.54679149
 -13.51459268  20.50110476]
[  2.61237085   5.79269127  10.66831179  10.10537749  20.57379756
 -13.40731184  20.74433195]


In [9]:
from feature_extractor import *
extractor = HOG()
image = cv2.imread('Untitled.png', cv2.IMREAD_COLOR)
ft = extractor.extract(image)
print(ft)

[[0.04687787]
 [0.07837174]
 [0.13816793]
 [0.31640455]
 [0.31640455]
 [0.17254841]
 [0.03771167]
 [0.02087594]
 [0.03490355]
 [0.0566633 ]
 [0.08689234]
 [0.14488405]
 [0.28522077]
 [0.31640455]
 [0.18477139]
 [0.04445079]
 [0.02782844]
 [0.03335017]
 [0.03107813]
 [0.0562946 ]
 [0.09884617]
 [0.31640455]
 [0.31640455]
 [0.22631736]
 [0.05736815]
 [0.03653723]
 [0.04571319]
 [0.05021108]
 [0.06413669]
 [0.08571801]
 [0.26098004]
 [0.31640455]
 [0.16158241]
 [0.04202996]
 [0.03720671]
 [0.03985576]]
